<a href="https://colab.research.google.com/github/ferkrum/web-scraper-imdb/blob/main/Web_scraper_IMDB_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtendo lista atualizada de lançamentos de filmes do IMDB 

Como obter uma lista atualizada com os últimos lançamentos de filmes do IMDB?

Fizemos um scraping usando a biblioteca Beautiful Soup e criamos um dataframe para visualização e exportação (arquivo CSV) dos resultados ao usuário. 


---
*Version history:
v2: adicionei o campo de data de lançamento ao dataframe italicized text*

In [ ]:
#Importa bibliotecas
import requests
from bs4 import BeautifulSoup

In [ ]:
# parsing
response = requests.get('https://www.imdb.com/calendar/?ref_=nv_mv_cal').content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
#verifica conteúdo do parser
soup

Abrindo a URL no navegador, inspecionamos o código e identificamos as tags relacionadas ao conteúdo buscado: div id=main

In [ ]:
lancamentos = soup.find("div",id="main")

In [ ]:
#confere conteudo filtrado em lancamentos
lancamentos


```
Notamos que para cada header 4, temos uma lista não numerada <ul>. 
Vamos usar essa estrutura para adicionar a coluna de data ao Data Frame.
```



In [ ]:
#buscando as datas dos lancamentos
datas = lancamentos.find_all("h4")

In [ ]:
#qtdade de datas diferentes 
len(datas)

60

In [ ]:
datas

In [ ]:
#fimes em cada data encontrada 
datasFilmes = lancamentos.find_all("ul")

In [ ]:
#confirma que é a mesma quantidade de datas vista celula anterior 
len(datasFilmes)

60

In [ ]:
#armazena quantidade de filmes
qtdadeFilmes = len(lancamentos.find_all("a"))

In [ ]:
#testando...
datasFilmes[59].find_all("a")

[<a href="/title/tt1462764/?ref_=rlm">Indiana Jones 5</a>,
 <a href="/title/tt7890826/?ref_=rlm">Naya Legend of the Golden Dolphin</a>]

In [ ]:
datas[0].text

'08 July 2022'

In [ ]:
for j in datasFilmes[0].find_all("a"):
  print(j.text)

Thor: Love and Thunder
Murina


In [ ]:
contaDatas = 0
indice = 0
dictLancamentos = {}
for i in datas:                                   #percorre as datas
  for j in datasFilmes[contaDatas].find_all("a"): #percorre todos itens para cada data
    indice += 1
    print (indice, " de ", qtdadeFilmes, end = ' ')
    print (i.text, end = ' ')
    
    response2 = requests.get("https://www.imdb.com"+j["href"]).content  #cria novo request para a url contida em cada elemento "href", ou seja, ira obter o conteúdo de cada página associada a cada novo filme da página
    soup2 = BeautifulSoup(response2, 'html.parser')                     #parser do conteúdo
    print("Dados do filme: ", j.text)
    sinopse = soup2.find("span",class_="sc-16ede01-2 gXUyNh").text      #armazena em sinopse o conteudo obtido em cada pagina
  
    dictLancamentos[indice] = {"nome" : j.text, "data lancamento" : i.text, " url" : "https://www.imdb.com"+j["href"], "sinopse" : sinopse}
  contaDatas+=1
#print(contaDatas)

1  de  100 08 July 2022 Dados do filme:  Thor: Love and Thunder
2  de  100 08 July 2022 Dados do filme:  Murina
3  de  100 13 July 2022 Dados do filme:  The Deer King
4  de  100 15 July 2022 Dados do filme:  Where the Crawdads Sing
5  de  100 15 July 2022 Dados do filme:  Paws of Fury: The Legend of Hank
6  de  100 15 July 2022 Dados do filme:  Mrs Harris Goes to Paris
7  de  100 22 July 2022 Dados do filme:  Nope
8  de  100 22 July 2022 Dados do filme:  How to Please a Woman
9  de  100 22 July 2022 Dados do filme:  Alone Together
10  de  100 22 July 2022 Dados do filme:  My Old School
11  de  100 29 July 2022 Dados do filme:  Thirteen Lives
12  de  100 29 July 2022 Dados do filme:  DC League of Super-Pets
13  de  100 29 July 2022 Dados do filme:  The Reef: Stalked
14  de  100 29 July 2022 Dados do filme:  Vengeance
15  de  100 29 July 2022 Dados do filme:  Resurrection
16  de  100 29 July 2022 Dados do filme:  Ali & Ava
17  de  100 05 August 2022 Dados do filme:  Bullet Train
18  de  

In [ ]:
import pandas as pd

In [ ]:
dfFilmes = pd.DataFrame.from_dict(dictLancamentos).T

In [ ]:
#exporta para CSV
pd.DataFrame.from_dict(dictLancamentos).T.to_csv('lancamentos.csv')

In [ ]:
dfFilmes

,nome,data lancamento,url,sinopse
1,Thor: Love and Thunder,08 July 2022,https://www.imdb.com/title/tt10648342/?ref_=rlm,"Thor enlists the help of Valkyrie, Korg and ex..."
2,Murina,08 July 2022,https://www.imdb.com/title/tt8434110/?ref_=rlm,A teenage girl decides to replace her controll...
3,The Deer King,13 July 2022,https://www.imdb.com/title/tt11417856/?ref_=rlm,The last survivor of a band of warriors is ens...
4,Where the Crawdads Sing,15 July 2022,https://www.imdb.com/title/tt9411972/?ref_=rlm,A woman who raised herself in the marshes of t...
5,Paws of Fury: The Legend of Hank,15 July 2022,https://www.imdb.com/title/tt4428398/?ref_=rlm,"Hank, a loveable dog with a head full of dream..."
...,...,...,...,...
96,Transformers: Rise of the Beasts,09 June 2023,https://www.imdb.com/title/tt5090568/?ref_=rlm,Plot unknown. Reportedly based on the 'Transfo...
97,Elemental,16 June 2023,https://www.imdb.com/title/tt15789038/?ref_=rlm,"Follows Ember and Wade, in a city where fire-,..."
98,The Flash,23 June 2023,https://www.imdb.com/title/tt0439572/?ref_=rlm,The plot is unknown. Feature film based on the...
99,Indiana Jones 5,30 June 2023,https://www.imdb.com/title/tt1462764/?ref_=rlm,The plot is unknown at this time.
